In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from ast import literal_eval

In [ ]:
%time datos = pd.read_csv('data_tokens.csv')

Wall time: 620 ms


In [ ]:
%time datos.tokens=datos.tokens.apply(literal_eval)

Wall time: 11.9 s


In [ ]:
tickets_cortos=pd.read_csv('tickets_cortos.csv')
similares = pd.read_csv("similares.csv")

In [ ]:
datos.drop(index=list(map(lambda elemento: elemento[0],list(tickets_cortos.values)+list(similares.values)))
           ,inplace=True
          )

In [ ]:
conteo_categorias=datos['etiquetas'].value_counts()
tabla_categorias=pd.DataFrame()
tabla_categorias['conteo']=conteo_categorias
tabla_categorias['porcentaje']=(conteo_categorias/datos.shape[0]*100)
tabla_categorias['acumulado']=tabla_categorias['porcentaje'].cumsum()
tabla_categorias

,conteo,porcentaje,acumulado
1,5312,32.370506,32.370506
2,3743,22.809263,55.179768
3,2932,17.867154,73.046923
0,2786,16.977453,90.024375
4,1637,9.975625,100.000000


In [ ]:
# filtro_otros=[]
# for nombre,grupo in datos.query("etiquetas==0").groupby('categorias'):
#     if nombre in ['Bswift']:
#         filtro_otros.append(grupo.loc[np.random.choice(grupo.index,550)])
#     else:
#         filtro_otros.append(grupo)
# filtro_otros=pd.concat(filtro_otros)

In [ ]:
# filtro_otros.categorias.value_counts().sum()

In [ ]:
conteo=datos.etiquetas.value_counts()
minimo=conteo.min()
cantidades=conteo.apply(lambda contador: minimo+contador*0.1 if contador!=minimo else contador).astype(int)
cantidades

1    2168
2    2011
3    1930
0    1915
4    1637
Name: etiquetas, dtype: int32

In [ ]:
cantidades.loc[0]=tabla_categorias.loc[0,'conteo']
cantidades.sort_values(ascending=False)

0    2786
1    2168
2    2011
3    1930
4    1637
Name: etiquetas, dtype: int64

In [ ]:
2786-1637

1149

In [ ]:
datos_equilibrados = pd.concat([grupo.loc[np.random.choice(grupo.index,cantidades.loc[nombre])] for nombre,grupo in datos.groupby('etiquetas')])

In [ ]:
# datos_equilibrados=datos_equilibrados.append(filtro_otros)

In [ ]:
# conteo_categorias=datos_equilibrados.query("etiquetas==0")['categorias'].value_counts()
# tabla_categorias=pd.DataFrame()
# tabla_categorias['conteo']=conteo_categorias
# tabla_categorias['porcentaje']=(conteo_categorias/datos_equilibrados.query("etiquetas==0").shape[0]*100)
# tabla_categorias['acumulado']=tabla_categorias['porcentaje'].cumsum()
# tabla_categorias

In [ ]:
conteo_categorias=datos_equilibrados['etiquetas'].value_counts()
tabla_categorias=pd.DataFrame()
tabla_categorias['conteo']=conteo_categorias
tabla_categorias['porcentaje']=(conteo_categorias/datos_equilibrados.shape[0]*100)
tabla_categorias['acumulado']=tabla_categorias['porcentaje'].cumsum()
tabla_categorias

,conteo,porcentaje,acumulado
0,2786,26.452716,26.452716
1,2168,20.584884,47.037600
2,2011,19.094189,66.131789
3,1930,18.325104,84.456893
4,1637,15.543107,100.000000


In [ ]:
datos_equilibrados.to_csv('data_equilibrada.csv',index=False)

# BoW:

In [ ]:
import nltk

In [ ]:
%time fdist = nltk.FreqDist(np.concatenate(datos_equilibrados.reset_index(drop=True)['tokens']))
%time tokens=fdist.most_common(len(fdist))
%time tokens_tf=pd.DataFrame(tokens,columns=['token','TF'])

Wall time: 3.78 s
Wall time: 60.6 ms
Wall time: 170 ms


In [ ]:
bow=tokens_tf.query("TF<2200 and TF>=160")['token']
#2200 como limite superior = muy cercano a la cantidad de tickets de la categoria 1
# Recordar que la categoria 0 es la categoria 'otros', que se compone de otras categorias

#160 es aproximadamente el 10% de los tickets de la categoria con menos tickets

In [ ]:
bow.to_csv('bow.csv',index=False)